This is a baseline model for DSSM (comparing with Triplet Hing Loss). It is using pointwise loss function, which is trained to increase the accuracy of classifying the user is interested in the item or not. Will update this model after the Triplet Hing Loss model is done.

In [39]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf


In [2]:
# Define the database folder path and file names
db_folder = '../data/processed_data/yelp_data/'
db_files = ['yelp_business_data.db', 'yelp_review_data.db', 'yelp_user_data.db', 'yelp_tip_data.db']
db_paths = [db_folder + db_file for db_file in db_files]

In [3]:
# Connect to the databases and load data
def load_data_from_db():
    data = {}
    
    # Open connections and read tables
    conns = [sqlite3.connect(db_path) for db_path in db_paths]
    try:
        # Load tables from the databases
        data['business'] = pd.read_sql_query("SELECT * FROM business_details", conns[0])
        data['categories'] = pd.read_sql_query("SELECT * FROM business_categories", conns[0])
        data['review'] = pd.read_sql_query("SELECT * FROM review_data", conns[1])
        data['user'] = pd.read_sql_query("SELECT * FROM user_data", conns[2])
        data['tip'] = pd.read_sql_query("SELECT * FROM tip_data", conns[3])
        
    finally:
        # Close all database connections
        for conn in conns:
            conn.close()

    return data

In [4]:
# Load data into a dictionary
yelp_data = load_data_from_db()

# Check loaded data
for table, df in yelp_data.items():
    print(f"Loaded {len(df)} rows from {table} table.")

Loaded 78059 rows from business table.
Loaded 360656 rows from categories table.
Loaded 980418 rows from review table.
Loaded 229447 rows from user table.
Loaded 173085 rows from tip table.


In [5]:
user_con_feature_lst = [
                        'review_count', 
                        'useful', 
                        'funny', 
                        'cool', 
                        'fans', 
                        'average_stars'
                        ]
business_con_feature_lst = [
                        'stars', 
                        'review_count', 
                        # 'latitude', 
                        # 'longitude'
                        ]

# add user features that start with 'compliment_'
user_compliment_feature_lst = [col for col in yelp_data['user'].columns if 'compliment_' in col]
user_con_feature_lst += user_compliment_feature_lst

In [6]:
# Preprocess user data
user_df = yelp_data['user']
user_df['yelping_since'] = pd.to_datetime(user_df['yelping_since'])

# Preprocess business data
business_df = yelp_data['business']
business_df['is_open'] = business_df['is_open'].fillna(0).astype(int)

# Preprocess review data
review_df = yelp_data['review']
# Create labels for review data
review_df['label'] = (review_df['stars'] >= 4).astype(int)

# Preprocess tip data
tip_df = yelp_data['tip']

# Preprocess categories data
categories_df = yelp_data['categories']


In [7]:
# Flatten all categories into a single list to fit the encoder
unique_categories = set([cat for sublist in categories_df['category'] for cat in sublist])
label_encoder = LabelEncoder()
label_encoder.fit(list(unique_categories))

categories_df['category_encoded'] = label_encoder.transform(categories_df['category'])

categories_grouped = categories_df.groupby('business_id')['category_encoded'].apply(list).reset_index()

# Merge the categories with the business data, name the column 'category_encoded'
business_df = business_df.merge(categories_grouped, on='business_id', how='left')

In [8]:
# Count the number of reviews and average review for each business
business_review_count = review_df.groupby('business_id').size()
business_avg_review = review_df.groupby('business_id')['stars'].mean()
business_df['review_count'] = business_review_count
business_df['avg_review'] = business_avg_review # similar to stars, but this is adjusted for the number of reviews extracted


In [9]:

# Example: Extract numerical features for embedding
user_features = user_df[user_con_feature_lst].fillna(0)
# Example: Extract numerical features
business_features = business_df[business_con_feature_lst].fillna(0)


In [10]:
# Encode user_id and business_id
user_id_encoder = LabelEncoder()
business_id_encoder = LabelEncoder()

user_df['user_id_encoded'] = user_id_encoder.fit_transform(user_df['user_id'])
business_df['business_id_encoded'] = business_id_encoder.fit_transform(business_df['business_id'])

# Save number of unique users and businesses for embedding input_dim
num_users = user_df['user_id_encoded'].max() + 1
num_businesses = business_df['business_id_encoded'].max() + 1



In [11]:
# # Example: Encode 'city' as a discrete feature for businesses
# business_city_encoder = LabelEncoder()
# business_df['city_encoded'] = business_city_encoder.fit_transform(business_df['city'])

# # Save number of unique cities for embedding input_dim
# num_cities = business_df['city_encoded'].max() + 1


In [12]:
# Standardize user continuous features
user_continuous_features = user_df[['review_count', 'useful', 'funny', 'cool', 'fans', 'average_stars']].fillna(0)
user_scaler = StandardScaler()
user_continuous_features_scaled = user_scaler.fit_transform(user_continuous_features)

# Standardize business continuous features
business_continuous_features = business_df[['stars', 'review_count', 'latitude', 'longitude']].fillna(0)
business_scaler = StandardScaler()
business_continuous_features_scaled = business_scaler.fit_transform(business_continuous_features)

# Ensure continuous features are pandas DataFrames
user_continuous_features_scaled = pd.DataFrame(user_continuous_features_scaled, index=user_df['user_id_encoded'])
business_continuous_features_scaled = pd.DataFrame(business_continuous_features_scaled, index=business_df['business_id_encoded'])

In [13]:
def create_embedding_layer(input_dim, output_dim, name):
    """Reusable function to create an embedding layer."""
    return layers.Embedding(input_dim=input_dim, output_dim=output_dim, name=f"{name}_embedding")

# Create embedding layers
user_id_embedding = create_embedding_layer(num_users, 16, "user_id")
business_id_embedding = create_embedding_layer(num_businesses, 16, "business_id")
# city_embedding = create_embedding_layer(num_cities, 8, "city")

# Define the number of unique categories
num_categories = len(label_encoder.classes_)
# Create an embedding layer for categories
category_embedding = create_embedding_layer(num_categories, 16, "category")

In [14]:
# Example aggregation function for TensorFlow (Mean Pooling)
def aggregate_category_embeddings(category_indices):
    category_indices = tf.constant(category_indices, dtype=tf.int32)
    embeddings = category_embedding(category_indices)
    return tf.reduce_mean(embeddings, axis=0)

In [15]:
# Filter out unseen user_id and business_id
review_df = review_df[
    (review_df['user_id'].isin(user_id_encoder.classes_)) & 
    (review_df['business_id'].isin(business_id_encoder.classes_))
]

# Encode user_id and business_id
review_df['user_id_encoded'] = user_id_encoder.transform(review_df['user_id'])
review_df['business_id_encoded'] = business_id_encoder.transform(review_df['business_id'])

C:\Users\yueny\AppData\Local\Temp\ipykernel_5656\1600752797.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['user_id_encoded'] = user_id_encoder.transform(review_df['user_id'])
C:\Users\yueny\AppData\Local\Temp\ipykernel_5656\1600752797.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['business_id_encoded'] = business_id_encoder.transform(review_df['business_id'])


In [16]:
def user_tower(continuous_dim):
    # Inputs
    user_id_input = layers.Input(shape=(1,), name="user_id")
    user_continuous_input = layers.Input(shape=(continuous_dim,), name="user_continuous")

    # Embedding
    user_id_embedded = user_id_embedding(user_id_input)
    user_id_embedded = layers.Flatten()(user_id_embedded)

    # Combine
    concat = layers.Concatenate()([user_id_embedded, user_continuous_input])
    x = layers.Dense(64, activation='relu')(concat)
    x = layers.Dense(32, activation='relu')(x)
    user_embedding = layers.Dense(16, activation=None, name="user_embedding")(x)

    return Model([user_id_input, user_continuous_input], user_embedding, name="UserTower")


In [17]:
def item_tower(continuous_dim):
    # Inputs
    business_id_input = layers.Input(shape=(1,), name="business_id")
    business_continuous_input = layers.Input(shape=(continuous_dim,), name="business_continuous")

    # Embedding
    business_id_embedded = business_id_embedding(business_id_input)
    business_id_embedded = layers.Flatten()(business_id_embedded)

    category_input = layers.Input(shape=(None,), dtype="int32", name="category_indices")  # Variable-length input
    category_embeddings = category_embedding(category_input)
    aggregated_category_embedding = layers.Lambda(lambda x: tf.reduce_mean(x, axis=1), name="category_pooling")(category_embeddings)

    # Combine
    concat = layers.Concatenate()([business_id_embedded, aggregated_category_embedding, business_continuous_input])

    x = layers.Dense(64, activation='relu')(concat)
    x = layers.Dense(32, activation='relu')(x)
    business_embedding = layers.Dense(16, activation=None, name="business_embedding")(x)

    return Model([business_id_input, category_input, business_continuous_input], business_embedding, name="ItemTower")


In [19]:
def safe_transform(encoder, values, default=-1):
    """
    Safely transform values using the encoder, assigning a default value for unseen labels.
    """
    try:
        return encoder.transform(values)
    except ValueError:
        # Use a mapping approach for unseen labels
        known_labels = set(encoder.classes_)
        transformed = [encoder.transform([v])[0] if v in known_labels else default for v in values]
        return np.array(transformed)

## The following part is for the Cosine Similarity and Binary Crossentropy. 

In [11]:
# Define cosine similarity layer
def cosine_similarity_layer():
    def cosine_sim(inputs):
        a, b = inputs
        a = tf.nn.l2_normalize(a, axis=-1)
        b = tf.nn.l2_normalize(b, axis=-1)
        return tf.reduce_sum(a * b, axis=-1, keepdims=True)
    return tf.keras.layers.Lambda(cosine_sim, name="CosineSimilarity")

In [33]:
# Instantiate towers
user_model = user_tower(user_continuous_features_scaled.shape[1])
item_model = item_tower(business_continuous_features_scaled.shape[1])

# Define inputs for user and business towers
user_inputs_model = [Input(shape=(1,), dtype=tf.int32, name="user_id_input"),
                     Input(shape=(user_continuous_features_scaled.shape[1],), name="user_cont_features_input")]

business_inputs_model = [Input(shape=(1,), dtype=tf.int32, name="business_id_input"),
                         Input(shape=(business_continuous_features_scaled.shape[1],), name="business_cont_features_input")]

# Get embeddings
user_embedding = user_model(user_inputs_model)
business_embedding = item_model(business_inputs_model)

# Compute cosine similarity
similarity = cosine_similarity_layer()([user_embedding, business_embedding])

# Define the full model
model = Model(inputs=user_inputs_model + business_inputs_model, outputs=similarity)


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

NameError: name 'cosine_similarity_layer' is not defined

In [ ]:
# Use indices from review_df to select rows
user_indices = review_df['user_id_encoded'].values
business_indices = review_df['business_id_encoded'].values

# Use .take or reindex based on indices
user_features = user_continuous_features_scaled.take(user_indices, axis=0)
business_features = business_continuous_features_scaled.take(business_indices, axis=0)

In [64]:
# Combine all inputs
all_user_inputs = [user_indices, user_features.values]
all_business_inputs = [business_indices, business_features.values]
all_labels = review_df['label'].values

# Split into training and testing sets
(
    train_user_indices, test_user_indices,
    train_business_indices, test_business_indices,
    train_user_features, test_user_features,
    train_business_features, test_business_features,
    train_labels, test_labels
) = train_test_split(
    user_indices, business_indices, user_features, business_features,
    all_labels, test_size=0.2, random_state=42
)

# Prepare training and testing inputs
train_user_inputs = [train_user_indices, train_user_features]
train_business_inputs = [train_business_indices, train_business_features]

test_user_inputs = [test_user_indices, test_user_features]
test_business_inputs = [test_business_indices, test_business_features]


In [47]:
model.fit(
    x=train_user_inputs + train_business_inputs,
    y=train_labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    verbose=1
)

Epoch 1/10
  379/19609 ━━━━━━━━━━━━━━━━━━━━ 4:31 14ms/step - accuracy: 0.6995 - loss: 0.6196

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(
    x=test_user_inputs + test_business_inputs,
    y=test_labels,
    verbose=1
)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

## The following part is for the Triplet Hinge Loss function. 

In [18]:
# Triplet loss function
def triplet_hinge_loss(margin=1.0):
    def loss(y_true, y_pred):
        # y_pred shape: (batch_size, 3, embedding_dim)
        anchor, positive, negative = tf.unstack(y_pred, num=3, axis=1)
        
        # Compute pairwise distances
        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        
        # Hinge loss: max(0, pos_dist - neg_dist + margin)
        return tf.reduce_mean(tf.maximum(pos_dist - neg_dist + margin, 0.0))
    return loss

In [19]:
# Function to generate triplets
def generate_triplets(review_df, num_neg_samples=1):
    triplets = []
    grouped = review_df.groupby('user_id_encoded')

    for user_id, group in grouped:
        positive_samples = group[group['label'] == 1]
        negative_samples = group[group['label'] == 0]
        
        if positive_samples.empty or negative_samples.empty:
            continue  # Skip users without both positive and negative samples
        
        for _, pos_row in positive_samples.iterrows():
            for _, neg_row in negative_samples.sample(num_neg_samples, replace=True).iterrows():
                triplets.append((
                    user_id,
                    pos_row['business_id_encoded'],
                    neg_row['business_id_encoded']
                ))
    
    return np.array(triplets)

In [ ]:
# Split review_df into train and test sets
train_df, test_df = train_test_split(review_df, test_size=0.2, random_state=42)

# Generate triplets for training and testing
train_triplets = generate_triplets(train_df)

In [ ]:
# Prepare train and test inputs
def prepare_triplet_inputs(triplets, user_features, business_features, business_category_map, max_category_length=5):
    # Replace NaN values with empty lists in `business_category_map`
    business_category_map = business_category_map.apply(lambda x: x if isinstance(x, list) else [])

    anchor_indices = triplets[:, 0]
    positive_indices = triplets[:, 1]
    negative_indices = triplets[:, 2]

    anchor_features = [anchor_indices, user_features.take(anchor_indices, axis=0).values]
    positive_features = [
        positive_indices,
        pad_sequences(business_category_map.loc[positive_indices].tolist(), maxlen=max_category_length, padding="post"),
        business_features.take(positive_indices, axis=0).values, 

]
    negative_features = [
        negative_indices, 
        pad_sequences(business_category_map.loc[negative_indices].tolist(), maxlen=max_category_length, padding="post"),
        business_features.take(negative_indices, axis=0).values,
    ]

    return [
        anchor_features[0], anchor_features[1],
        positive_features[0], positive_features[1], positive_features[2],
        negative_features[0], negative_features[1], negative_features[2]
    ]

business_category_map = business_df.set_index('business_id_encoded')['category_encoded']


train_inputs = prepare_triplet_inputs(train_triplets, user_continuous_features_scaled, business_continuous_features_scaled, business_category_map)

In [33]:
# Instantiate towers
user_model = user_tower(user_continuous_features_scaled.shape[1])
item_model = item_tower(business_continuous_features_scaled.shape[1])

# Define inputs for user and business towers
user_inputs_model = [Input(shape=(1,), dtype=tf.int32, name="user_id_input"),
                     Input(shape=(user_continuous_features_scaled.shape[1],), name="user_cont_features_input")]

positive_inputs_model = [
    Input(shape=(1,), dtype=tf.int32, name="positive_id_input"),
    Input(shape=(None,), dtype=tf.int32, name="positive_category_input"),
    Input(shape=(business_continuous_features_scaled.shape[1],), name="positive_cont_features_input")
]

negative_inputs_model = [
    Input(shape=(1,), dtype=tf.int32, name="negative_id_input"),
    Input(shape=(None,), dtype=tf.int32, name="negative_category_input"),
    Input(shape=(business_continuous_features_scaled.shape[1],), name="negative_cont_features_input")
]

# Generate embeddings
anchor_embedding = user_model(user_inputs_model)
positive_embedding = item_model(positive_inputs_model)
negative_embedding = item_model(negative_inputs_model)

In [34]:
def stack_embeddings(embeddings):
    # Unpack the embeddings from the list
    anchor, positive, negative = embeddings
    return tf.stack([anchor, positive, negative], axis=1)

triplet_embeddings = Lambda(stack_embeddings, name="triplet_embeddings")(
    [anchor_embedding, positive_embedding, negative_embedding]
)

In [37]:
all_inputs = user_inputs_model + positive_inputs_model + negative_inputs_model
# Build the model
triplet_model = Model(
    inputs= all_inputs,
    outputs=triplet_embeddings,
    name="triplet_model"
)

# Compile with triplet loss
triplet_model.compile(
    optimizer='adam',
    loss=triplet_hinge_loss(margin=0.1)
)

In [38]:
# Train the triplet model
triplet_model.fit(
    x=train_inputs,
    y=np.zeros(len(train_inputs[0])),  # Dummy labels, as loss is computed from embeddings
    batch_size=32,
    epochs=10,
    verbose=1
)


Epoch 1/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 179s 14ms/step - loss: 0.0180
Epoch 2/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 180s 14ms/step - loss: 0.0141
Epoch 3/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 189s 15ms/step - loss: 0.0109
Epoch 4/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 196s 15ms/step - loss: 0.0087
Epoch 5/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 196s 15ms/step - loss: 0.0069
Epoch 6/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 196s 15ms/step - loss: 0.0056
Epoch 7/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 214s 17ms/step - loss: 0.0048
Epoch 8/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 207s 16ms/step - loss: 0.0041
Epoch 9/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 196s 15ms/step - loss: 0.0036
Epoch 10/10
12867/12867 ━━━━━━━━━━━━━━━━━━━━ 207s 16ms/step - loss: 0.0032


In [40]:
# Function to get a user embedding
def get_user_embedding(user_id, user_features):
    user_input = [
        np.array([user_id]),  # user_id_encoded as an array (needs to be a batch)
        np.array([user_features])  # corresponding continuous features for the user
    ]
    user_embedding = user_model.predict(user_input)
    return user_embedding

# Function to get a business embedding
def get_business_embedding(business_id, business_features, business_category_map):
    business_input = [
        np.array([business_id]),  # business_id_encoded as an array (needs to be a batch)
        np.array([business_category_map.get(business_id, [])]),  # business categories
        np.array([business_features])  # corresponding continuous features for the business
    ]
    business_embedding = item_model.predict(business_input)
    return business_embedding

In [41]:

# Function to calculate similarity
def calculate_similarity(user_id, business_id, user_features, business_features, business_category_map):
    # Get user and business embeddings
    user_emb = get_user_embedding(user_id, user_features)
    business_emb = get_business_embedding(business_id, business_features, business_category_map)

    # Calculate cosine similarity
    similarity = cosine_similarity(user_emb, business_emb)
    return similarity[0][0]  # Since cosine_similarity returns a 2D array

In [42]:
# calculate similarity in the test_df using the triplet model, compare with the actual label

# Get user embeddings
user_embeddings = user_model.predict([test_user_indices, test_user_features])

# Get business category for each business in the test set
test_business_categories = business_category_map.loc[test_business_indices].tolist()

# Get business embeddings
business_embeddings = item_model.predict([
    test_business_indices,
    pad_sequences(test_business_categories, maxlen=5, padding="post"),
    test_business_features
])

# Calculate cosine similarity
similarities = cosine_similarity(user_embeddings, business_embeddings)

# Calculate accuracy
predictions = (similarities > 0).astype(int)
accuracy = np.mean(predictions == test_labels)
print(f"Accuracy: {accuracy}")

NameError: name 'test_user_indices' is not defined

In [177]:
# Calculate accuracy
def calculate_accuracy(similarities):
    correct = 0
    # calculate the number of correct predictions, if positive similarity is greater than 0 and negative similarity is less than 0, add the count
    for pos, neg in similarities:
        if pos > 0:
            correct += 1
        if neg < 0:
            correct += 1
    return correct / (2 * len(similarities))

accuracy = calculate_accuracy(similarities)
print(f"Accuracy: {accuracy}")

Accuracy: 0.55
